# Tracking habits with todoist
> get completed items from todoist
- toc: true 
- badges: true
- comments: true 
- author: Dan Hosanee
- categories: [python]


## Background 

This project essentially ultises the todoist api to track completed items. If you have not heard of todoist check it out [here](https://todoist.com/).

## Why 

To have the ability to track my habits over time. While todoist does not provide this functionality out of the box we can acheive this via its [api](https://developer.todoist.com/sync/). This is in the context of setting up reoccuring tasks.

## Notes 

- There is an official [python client](https://github.com/Doist/todoist-api-python) however the sync api provides more options in terms of requests you can make

- The request will be focusing on the [get all completed items](https://developer.todoist.com/sync/v9/#get-all-completed-items) which is only avaliable on the sync api

- There is a limit of 200 items to be returned, therefore plan is to do a weekly extraction from Monday to Sunday AEST

- Timezones do play a factor here the api takes and returns date parameters in UTC

- At the moment the python script will output a csv file, however the plan is to persist the data either within AWS enviornment OR SQLite etc. 

- Script will be scheduled to be run ideally every monday


## To Do 

- ~~[x] Write python script to get completed items from the previous week~~

- ~~[x] Create function to convert local timezone to UTC~~ 

- ~~[x] Convert Timezone response back to AEST in the csv output~~

- [ ] Method of scheduling script: AWS lambda, Airflow etc.

- [ ] Method of persisting date: S3, Aurora, SQLite etc. 


## Learnings 

- Focus on making functions pure

- Perhaps the script could be better separated


## [Python Script](https://github.com/danhosanee/todoist-pipeline/blob/main/get_completed.py)

```python

import json
import requests
from urllib.parse import urljoin
from typing import Dict
import configparser
import pandas as pd
from datetime import datetime
from datetime import timezone


BASE_URL = "https://api.todoist.com"
AUTH_BASE_URL = "https://todoist.com"
SYNC_VERSION = "v9"
SYNC_API = urljoin(BASE_URL, f"/sync/{SYNC_VERSION}/")
AUTHORIZATION = ("Authorization", "Bearer %s")


def create_headers(token: str) -> Dict[str, str]:
    headers: Dict[str, str] = {}
    headers.update([(AUTHORIZATION[0], AUTHORIZATION[1] % token)])
    return headers


def get_sync_url(relative_path: str) -> str:
    return urljoin(SYNC_API, relative_path)


def request_string(request_string: str, **kwargs) -> str:
    end_point = get_sync_url(request_string)
    filters = "".join(f"&{k}={v}"for k, v in kwargs.items())
    return f"{end_point}?{filters}"


def get_request(url: str, header: str) -> json:
    try:
        r = requests.get(url, headers=header)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    return r.json()


def date_offset(date_input: datetime,  week_num: int, week_day: int) -> datetime:
    date = date_input + pd.tseries.offsets.Week(week_num, weekday=week_day)
    return date


def format_str(date: datetime, strformat: str) -> str:
    return date.strftime(strformat)


def json_to_df(json: json, path: list) -> pd.DataFrame():
    return pd.json_normalize(json, record_path=path)


def tz_to_utc(date: datetime) -> str:
    return date.tz_convert(timezone.utc).strftime("%Y-%m-%dT%H:%M")


def main():

    config = configparser.ConfigParser()

    config.read("settings.ini")

    request_header = create_headers(config["API_CONFIG"]["API_CODE"])

    get_tz_url = get_sync_url("user")

    get_tz_request = get_request(url=get_tz_url, header=request_header)

    tz = get_tz_request['tz_info']['timezone']

    date_today = datetime.now().date()

    mon_week_adj = -2 if date_today != 0 else -1

    prev_monday = date_offset(date_input=date_today,
                              week_num=mon_week_adj, week_day=0).tz_localize(tz)

    prev_sunday = date_offset(date_input=date_today,
                              week_num=-1, week_day=6).tz_localize(tz)

    get_completed_url = request_string(
        "completed/get_all", since=tz_to_utc(prev_monday), until=tz_to_utc(prev_sunday)
    )

    get_completed_request = get_request(
        url=get_completed_url, header=request_header)

    df_items = json_to_df(get_completed_request, ['items'])

    df_trnsform = (
        df_items
        .assign(
            completed_at=pd.to_datetime(df_items['completed_at'])
            .dt.tz_convert(tz)
            .dt.strftime('%Y-%m-%d %H:%M')
        )
        .to_csv(f"completedItems_{prev_sunday.date()}.csv", index=False)
    )


if __name__ == "__main__":
    main()
```